In [ ]:
using InspectDR
using Colors

#Define colour constants:
black = RGB24(0, 0, 0); white = RGB24(1, 1, 1)
red = RGB24(1, 0, 0); green = RGB24(0, 1, 0); blue = RGB24(0, 0, 1)

#Reset list of displays for demo purposes (in case GtkDisplay is currently active)
function resetdisplays()
    dfltdisp = Base.Multimedia.displays[end]
    if typeof(dfltdisp) <: InspectDR.GtkDisplay
        pop!(Base.Multimedia.displays)
    end
    nothing
end

nothing

# Define "plot" object

In [ ]:
#Modify previously-generated Bode plot:
#fmin, fmax, p0: in Hz
function PlotBode(plot, fmin, fmax, p0)
    const style = :solid
    const npts = 100
    f = logspace(log10(fmin), log10(fmax), npts)
    y = 1./(1-(im*f/p0))
    plot.data = [] #Clear old data
    wfrm = add(plot, f, y)
    wfrm.line = line(color=blue, width=1, style=style)
    plot.ext_full = InspectDR.PExtents2D(NaN, NaN, -40, 0)
    return plot
end

#Create new Bode plot:
function PlotBode(fmin, fmax, p0)
    const w = 500; const h = w/1.6 #Select plot width/height
    plot = InspectDR.Plot2D()
    plot.axes = InspectDR.axes(:log10, :dB20)
    plot.layout.grid = grid(vmajor=true, vminor=true, hmajor=false)
    plot.layout.wdata=w; plot.layout.hdata=h
    a = plot.annotation
    a.title = "Bode Plot"
    a.xlabel = "Frequency (Hz)"
    a.ylabel = "Amplitude (dB)"
    return PlotBode(plot, fmin, fmax, p0)
end

plot = PlotBode(1e6, 20e9, 0.5e9)
println("\"plot\" defined.")
nothing

# Test inline SVG plots
**NOTE:** SVG outputs do not render well in notebooks for some reason...

In [ ]:
#Jupyter typically requests inline graphics as SVG first.
#Allowing MIME"image/svg+xml" renderings therefore produces SVG outputs:
InspectDR.defaults.rendersvg = true

resetdisplays() #Just in case
plot #Implicit call to "optimal" "show(..,::MIME,...)" function

# Test inline PNG plots

In [ ]:
#When MIME"image/svg+xml" is disabled, Jupyter eventually requests PNG inline graphics:
InspectDR.defaults.rendersvg = false

resetdisplays() #Just in case
display(plot) #Explicit call to display plot
nothing

# Test InspectDR Gtk "Display"

In [ ]:
resetdisplays() #Just in case
pushdisplay(InspectDR.GtkDisplay()) #Make "GtkDisplay" highest priority
display(plot) #This time InspectDR.GtkDisplay() is on top of display stack
nothing

# Now test display() with explicit GtkDisplay<:Display object

In [ ]:
resetdisplays()
display(Base.Multimedia.displays) #GtkDisplay should not longer be on display stack
display(InspectDR.GtkDisplay(), plot) #Explicit call to use Gtk "display"
nothing

# Save plot to .svg file

In [ ]:
InspectDR.write_svg("plotsave.svg", plot) #High-quality output!

# Demo complete!